In [2]:
!pip install xgboost

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [8]:
import pandas as pd
import numpy as np

np.random.seed(42)

N = 15000

# Generamos ingresos realistas
ingreso = np.random.uniform(400_000, 5_000_000, N)

# Antigüedad laboral
antiguedad = np.random.uniform(0, 120, N)

# Generamos ratio de gasto controlado
ratio = np.random.uniform(0.3, 1.1, N)
gastos = ingreso * ratio

margen = ingreso - gastos

# NUEVAS reglas más equilibradas
label = np.select(
    [
        (ratio >= 0.85) | (margen < 150_000),
        (ratio >= 0.60) & (ratio < 0.85),
        (ratio < 0.60) & (antiguedad >= 12),
    ],
    [0, 1, 2],
    default=1
)

df = pd.DataFrame({
    "ingreso_mensual": ingreso.round(0).astype(int),
    "gastos_mensuales": gastos.round(0).astype(int),
    "antiguedad_laboral_meses": antiguedad.round(0).astype(int),
    "label": label.astype(int)
})

print(df["label"].value_counts())

df.to_csv("credit_data.csv", index=False)
print("Dataset regenerado correctamente")

label
1    5097
2    5080
0    4823
Name: count, dtype: int64
Dataset regenerado correctamente


In [9]:

df = pd.read_csv("/content/credit_data.csv")
df = df.dropna()

X = df[["ingreso_mensual", "gastos_mensuales", "antiguedad_laboral_meses"]]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    random_state=42
)

model.fit(X_train, y_train)

pred = model.predict(X_test)
print(classification_report(y_test, pred))

joblib.dump(model, "credit_model_xgb.joblib")

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       965
           1       0.96      0.95      0.96      1019
           2       0.98      0.99      0.98      1016

    accuracy                           0.97      3000
   macro avg       0.97      0.97      0.97      3000
weighted avg       0.97      0.97      0.97      3000



['credit_model_xgb.joblib']

In [13]:
import joblib
import numpy as np
import pandas as pd

MODEL_PATH = "credit_model_xgb.joblib"

LABEL_MAP = {
    0: {"status": "NO_CUMPLE_SOBREENDEUDAMIENTO", "monto_maximo": 0},
    1: {"status": "CUMPLE_PARCIAL_MONTO_BAJO", "monto_maximo": 1_000_000},
    2: {"status": "CUMPLE_MONTO_ALTO", "monto_maximo": 10_000_000},
}

def explain_pred(label: int) -> dict:
    return LABEL_MAP.get(int(label), {"status": "DESCONOCIDO", "monto_maximo": None})

def predict_one(model, ingreso, gastos, antig_meses):
    X = np.array([[ingreso, gastos, antig_meses]], dtype=float)
    label = int(model.predict(X)[0])

    # Probabilidades si están disponibles
    proba = None
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[0].tolist()

    return {
        "input": {
            "ingreso_mensual": ingreso,
            "gastos_mensuales": gastos,
            "antiguedad_laboral_meses": antig_meses,
        },
        "label": label,
        "decision": explain_pred(label),
        "proba": proba,
    }

def main():
    print(f"Cargando modelo: {MODEL_PATH}")
    model = joblib.load(MODEL_PATH)
    print("OK ✅\n")

    # === 1) Casos manuales ===
    cases = [
        # Malo (gastos muy altos vs ingreso)
        (800_000, 760_000, 8),
        (1_200_000, 1_100_000, 24),

        # Límite / medio
        (900_000, 600_000, 6),
        (1_100_000, 750_000, 10),

        # Bueno (margen alto + antigüedad)
        (1_800_000, 650_000, 36),
        (3_000_000, 1_000_000, 60),
    ]

    print("=== Predicción casos manuales ===")
    for ingreso, gastos, antig in cases:
        out = predict_one(model, ingreso, gastos, antig)
        print(out)
    print()

    # === 2) Batch test ===
    df_test = pd.DataFrame({
        "ingreso_mensual": [500_000, 900_000, 1_500_000, 2_500_000],
        "gastos_mensuales": [480_000, 650_000, 700_000, 1_100_000],
        "antiguedad_laboral_meses": [2, 8, 18, 48],
    })

    X_batch = df_test[["ingreso_mensual", "gastos_mensuales", "antiguedad_laboral_meses"]].to_numpy(dtype=float)

    preds = model.predict(X_batch).astype(int)
    df_test["label_pred"] = preds
    df_test["status_pred"] = [explain_pred(x)["status"] for x in preds]
    df_test["monto_maximo_pred"] = [explain_pred(x)["monto_maximo"] for x in preds]

    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X_batch)
        # guarda como columnas proba_0, proba_1, proba_2 (si existen)
        for i in range(proba.shape[1]):
            df_test[f"proba_{i}"] = proba[:, i]

    print("=== Batch test (tabla) ===")
    print(df_test.to_string(index=False))
    print()

    # === 3) Chequeo rápido de clases soportadas ===
    # XGBoost normalmente tiene .classes_
    classes = getattr(model, "classes_", None)
    print("Clases detectadas en el modelo:", classes)

    print("\nFIN ✅")

if __name__ == "__main__":
    main()

Cargando modelo: credit_model_xgb.joblib
OK ✅

=== Predicción casos manuales ===
{'input': {'ingreso_mensual': 800000.54, 'gastos_mensuales': 760000, 'antiguedad_laboral_meses': 8}, 'label': 0, 'decision': {'status': 'NO_CUMPLE_SOBREENDEUDAMIENTO', 'monto_maximo': 0}, 'proba': [0.8457189202308655, 0.1537410020828247, 0.0005400601658038795]}
{'input': {'ingreso_mensual': 1200000, 'gastos_mensuales': 1100000.25, 'antiguedad_laboral_meses': 24}, 'label': 0, 'decision': {'status': 'NO_CUMPLE_SOBREENDEUDAMIENTO', 'monto_maximo': 0}, 'proba': [0.7957800626754761, 0.1915128529071808, 0.012707102112472057]}
{'input': {'ingreso_mensual': 900000, 'gastos_mensuales': 600000, 'antiguedad_laboral_meses': 6}, 'label': 1, 'decision': {'status': 'CUMPLE_PARCIAL_MONTO_BAJO', 'monto_maximo': 1000000}, 'proba': [0.018862903118133545, 0.9748913645744324, 0.006245742551982403]}
{'input': {'ingreso_mensual': 1100000, 'gastos_mensuales': 750000, 'antiguedad_laboral_meses': 10}, 'label': 1, 'decision': {'stat

In [11]:
import pandas as pd

df = pd.read_csv("/content/credit_data.csv")
print(df["label"].value_counts(dropna=False))
print("Clases:", sorted(df["label"].unique()))
print("Filas:", len(df))

label
1    5097
2    5080
0    4823
Name: count, dtype: int64
Clases: [np.int64(0), np.int64(1), np.int64(2)]
Filas: 15000
